In [5]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.2 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats
from scipy.stats import uniform

In [7]:
df = pd.read_csv('train.csv', index_col=0)
df

,full_sq,floor,build_year,num_room,kitch_sq,state,product_type,raion_popul,indust_part,sport_objects_raion,...,cafe_count_1000_price_high,cafe_sum_1500_min_price_avg,green_part_2000,cafe_sum_2000_min_price_avg,mosque_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,year,price_doc
0,43,24.0,2014.0,1.0,10.0,1.0,1,8.294300,0.007122,0,...,0,6.907755,3.710396,6.725430,0,1.803359,6.616560,0,2014,5533460
1,66,3.0,1979.0,3.0,10.0,3.0,0,11.314231,0.232205,4,...,0,6.626122,2.764431,6.479615,1,2.631169,6.469948,1,2013,9900000
2,55,5.0,1979.0,2.0,6.0,2.0,0,11.718817,0.000170,5,...,0,6.368753,3.511844,6.192138,0,1.363537,6.462171,0,2012,1990000
3,45,7.0,1970.0,2.0,6.0,2.0,0,11.331032,0.428826,4,...,0,6.753601,1.814825,6.551080,0,2.732418,6.458229,0,2014,6500000
4,43,4.0,1968.0,2.0,6.0,2.0,0,11.538769,0.136330,6,...,0,6.636603,2.257588,6.646507,1,2.020222,6.634778,2,2014,8500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21324,56,19.0,1979.0,2.0,1.0,1.0,1,11.655744,0.049637,6,...,0,6.492240,2.711378,6.816188,0,3.297317,6.536155,0,2014,10159562
21325,42,5.0,1977.0,1.0,8.0,3.0,0,11.718817,0.000170,5,...,0,6.502295,3.802208,6.291569,0,1.442202,6.487684,0,2014,5100000
21326,32,3.0,1979.0,2.0,6.0,2.0,0,11.719940,0.000000,4,...,0,6.572814,3.062456,6.583520,1,2.733718,6.573904,2,2011,4800000
21327,43,4.0,1969.0,2.0,5.0,2.0,0,11.331032,0.428826,4,...,0,6.586172,2.408745,6.563982,0,3.034953,6.476311,0,2013,6300000


In [8]:
df.shape

(21329, 51)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21329 entries, 0 to 21328
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   full_sq                      21329 non-null  int64  
 1   floor                        21329 non-null  float64
 2   build_year                   21329 non-null  float64
 3   num_room                     21329 non-null  float64
 4   kitch_sq                     21329 non-null  float64
 5   state                        21329 non-null  float64
 6   product_type                 21329 non-null  int64  
 7   raion_popul                  21329 non-null  float64
 8   indust_part                  21329 non-null  float64
 9   sport_objects_raion          21329 non-null  int64  
 10  shopping_centers_raion       21329 non-null  int64  
 11  radiation_raion              21329 non-null  int64  
 12  build_count_block            21329 non-null  float64
 13  build_count_brick    

In [88]:
# Загрузка данных
df = pd.read_csv('train.csv')


# Удаление незначимых столбцов
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Удаление дубликатов
df = df.drop_duplicates()

# Обработка пропущенных значений
df = df.dropna()

# Добавление новых фичей
df['ave_room_sq'] = df['full_sq'] / df['num_room']
df['ave_sport_km'] = (df['fitness_km'] + df['stadium_km'] + df['basketball_km']) / 3
df['ave_culture_km'] = (df['theater_km'] + df['exhibition_km']) / 2
df['build_material_idx'] = df['build_count_brick'] / df['build_count_block']

df.drop(columns=['fitness_km', 'stadium_km', 'basketball_km', 'theater_km', 'exhibition_km'], inplace=True)

# Определение признаков и целевой переменной
X = df.drop(columns=['price_doc'])
y = df['price_doc']

# # Обработка выбросов
# z_scores = np.abs(stats.zscore(X.select_dtypes(include=[np.number])))
# X = X[(z_scores < 3).all(axis=1)]
# y = y.loc[X.index]


X



,full_sq,floor,build_year,num_room,kitch_sq,state,product_type,raion_popul,indust_part,sport_objects_raion,...,cafe_sum_2000_min_price_avg,mosque_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,year,ave_room_sq,ave_sport_km,ave_culture_km,build_material_idx


In [87]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение трансформеров
# numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
# categorical_features = X.select_dtypes(include=[object]).columns.tolist()

# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# Применение предобработки к данным
# X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_test_preprocessed = preprocessor.transform(X_test)

# Первоначальная модель для получения важности признаков
initial_model = CatBoostRegressor(iterations=100, eval_metric='RMSE', logging_level='Silent')
initial_model.fit(X_train, y_train)

initial_model.get_feature_importance(prettified = True)

,Feature Id,Importances
0,full_sq,34.083964
1,ave_sport_km,9.427000
2,ttk_km,8.526645
3,build_year,2.853548
4,cafe_sum_1500_min_price_avg,2.530069
5,num_room,2.299156
6,ave_culture_km,2.268793
7,mkad_km,2.167386
8,indust_part,2.158550
9,floor,2.095343


In [10]:
# Загрузка данных
df = pd.read_csv('train.csv')

# Удаление незначимых столбцов
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Удаление дубликатов
df = df.drop_duplicates()

# Обработка пропущенных значений
df = df.dropna()

# Определение признаков и целевой переменной
X = df.drop(columns=['price_doc'])
y = df['price_doc']

# Обработка выбросов
for feature in X.select_dtypes(include=[np.number]).columns:
    z_scores = np.abs(stats.zscore(X[feature]))
    X = X[z_scores < 3]
    y = y[X.index]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение трансформеров
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=[object]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Применение предобработки к данным
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Первоначальная модель для получения важности признаков
initial_model = CatBoostRegressor(iterations=100, eval_metric='RMSE', logging_level='Silent')
initial_model.fit(X_train_preprocessed, y_train)

# Получение важности признаков
feature_importances = initial_model.get_feature_importance(prettified=True)
top_features_indices = feature_importances.nlargest(20, 'Importances')['Feature Id'].tolist()

# Преобразование индексов в имена признаков
top_features = [numeric_features[int(idx)] for idx in top_features_indices]

# Выбор топ-20 признаков
X = df[top_features]
y = df['price_doc']

# Обработка выбросов для топ-20 признаков
for feature in X.select_dtypes(include=[np.number]).columns:
    z_scores = np.abs(stats.zscore(X[feature]))
    X = X[z_scores < 3]
    y = y[X.index]

# Разделение данных на обучающую и тестовую выборки для топ-20 признаков
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение предобработки к данным для топ-20 признаков
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=[object]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Опциональная настройка параметров с помощью RandomizedSearchCV
param_dist = {
    'depth': [4, 6, 8, 10],
    'learning_rate': uniform(0.01, 0.1),
    'iterations': [100, 200, 300, 500]
}

random_search = RandomizedSearchCV(estimator=CatBoostRegressor(early_stopping_rounds=50, eval_metric='RMSE', logging_level='Silent'),
                                   param_distributions=param_dist, n_iter=20, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
random_search.fit(X_train_preprocessed, y_train)

best_model = random_search.best_estimator_

# Запуск кросс-валидации
cv_params = {
    'iterations': best_model.get_params()['iterations'],
    'depth': best_model.get_params()['depth'],
    'learning_rate': best_model.get_params()['learning_rate'],
    'loss_function': 'RMSE',
    'logging_level': 'Silent'
}

train_pool = Pool(X_train_preprocessed, y_train)
cv_data = cv(
    params=cv_params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42,
    stratified=False,
    plot=True
)

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test_preprocessed)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод результатов
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

# Обработка тестового набора данных
test_df = pd.read_csv('test.csv', index_col=0)

# Проверка наличия всех необходимых признаков в тестовом наборе данных
missing_features = [feature for feature in top_features if feature not in test_df.columns]
if missing_features:
    raise KeyError(f"Отсутствуют признаки в тестовом наборе: {', '.join(missing_features)}")

# Сортировка признаков в тестовом наборе данных в том же порядке, что и в тренировочном наборе данных
test_df = test_df[top_features]

# Применение тех же трансформаций, что и для тренировочного набора данных
test_df_preprocessed = preprocessor.transform(test_df)

# Генерация предсказаний
test_preds = best_model.predict(test_df_preprocessed)

# Создание файла для сабмита
submission = pd.DataFrame({'id': test_df.index, 'price_doc': test_preds})
submission.to_csv('submission.csv', index=False)

KeyboardInterrupt: 

In [ ]:
# Подготовка данных
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Предварительная обработка данных
# Пример: удаление пропусков (если есть)
X = X.dropna()
y = y[X.index]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выделение пула данных под CatBoost
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Опциональная настройка параметров с помощью RandomizedSearchCV
param_dist = {
    'depth': [4, 6, 8, 10],
    'learning_rate': uniform(0.01, 0.1),
    'iterations': [100, 200, 300, 500]  # Увеличено количество итераций
}

random_search = RandomizedSearchCV(
    estimator=CatBoostRegressor(eval_metric='RMSE', logging_level='Silent'),
    param_distributions=param_dist,
    n_iter=20,  # Увеличено количество итераций
    cv=5,  # Увеличено количество фолдов для более точной оценки
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

# Запуск кросс-валидации
cv_params = {
    'iterations': 500,
    'depth': best_model.get_params()['depth'],
    'learning_rate': best_model.get_params()['learning_rate'],
    'loss_function': 'RMSE',
    'logging_level': 'Silent'
}

cv_data = cv(
    params=cv_params,
    pool=train_pool,
    fold_count=5,  # Увеличено количество фолдов для более точной оценки
    shuffle=True,
    partition_random_seed=42,
    stratified=False,
    plot=True
)

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод результатов
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")


In [ ]:
# Подготовка данных
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выделение пула данных под CatBoost
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Настройка параметров модели
model_params = {
    'iterations': 500,  # Уменьшено количество итераций для ускорения
    'depth': 6,
    'learning_rate': 0.1,
    'early_stopping_rounds': 50,
    'eval_metric': 'RMSE',
    'logging_level': 'Verbose'
}

# Создание и обучение модели CatBoost с кросс-валидацией
model = CatBoostRegressor(**model_params)
model.fit(train_pool, eval_set=test_pool, verbose=False)

# Опциональная настройка параметров с помощью RandomizedSearchCV
param_dist = {
    'depth': [4, 6, 8, 10],
    'learning_rate': uniform(0.01, 0.1),
    'iterations': [100, 200, 300]  # Уменьшено количество итераций для ускорения
}

random_search = RandomizedSearchCV(estimator=CatBoostRegressor(), param_distributions=param_dist, n_iter=10, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

# Запуск кросс-валидации
cv_params = {
    'iterations': 500,  # Уменьшено количество итераций для ускорения
    'depth': 6,
    'learning_rate': 0.1,
    'loss_function': 'RMSE',
    'logging_level': 'Silent'
}

cv_data = cv(
    params=cv_params,
    pool=train_pool,
    fold_count = 5,  # Уменьшено количество фолдов для ускорения
    shuffle=True,
    partition_random_seed=42,
    stratified=False,
    plot=True
)

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод результатов
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

In [ ]:
cv_data

In [ ]:
y_pred.mean()

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
feature_importances = model.get_feature_importance(prettified=True)
df_feature_importances = pd.DataFrame(feature_importances)
new_features = df_feature_importances['Feature Id'].tolist()

In [ ]:
df

In [ ]:
X = df[new_features]
Y = df['price_doc']

# Предварительная обработка данных
X = X.dropna()
y = y[X.index]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выделение пула данных под CatBoost
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Сетка параметров для поиска
param_grid = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [100, 200, 300]
}

# Используем GridSearchCV для поиска наилучших параметров
grid_search = GridSearchCV(
    estimator=CatBoostRegressor(eval_metric='RMSE', logging_level='Silent'),
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод результатов
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

# Проверка распределения остатков
residuals = y_test - y_pred
plt.hist(residuals, bins=50)
plt.title("Histogram of Residuals")
plt.show()

# Визуализация важности признаков
feature_importances = best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]
features = X.columns[indices]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), feature_importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), features, rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
X = df[new_features]
y = df['price_doc']

# Предварительная обработка данных
X = X.dropna()
y = y[X.index]

categorical_features = ['state', 'product_type', 'radiation_raion']
for cat_col in categorical_features:
    X[cat_col] = X[cat_col].astype(str)

pool = Pool(data=X, label=y, cat_features=categorical_features)

params = {
    'iterations': 1000,
    'depth': 6,
    'learning_rate': 0.1,
    'loss_function': 'RMSE'
}

cv_data = cv(pool, params, fold_count=5, plot=True)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выделение пула данных под CatBoost
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

model = CatBoostRegressor(**params)
model.fit(train_pool)

# Предсказание на тестовых данных
y_pred = model.predict(test_pool)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод результатов
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

In [ ]:
submission = pd.read_csv('submission.csv', index_col=0)

In [ ]:
submission

In [ ]:
y_pred

In [ ]:
test_df = pd.read_csv('test.csv', index_col=0)

test_preds = best_model.predict(test_df[X.columns])
submission['price_doc'] = test_preds
print(len(test_preds))
print(submission.shape)
submission.to_csv('submission.csv')